# Pneumonia Prediction Model - Exploratory Data Analysis

This notebook demonstrates how to use the pneumonia prediction model and explore its capabilities.

In [ ]:
# Import required libraries
import sys
import os
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO

# Import our modules
from src.models.model import PneumoniaModel
from src.inference.predictor import PneumoniaPredictor
from src.models.utils import preprocess_image, resize_image

In [ ]:
# Initialize the model
model = PneumoniaModel()
model.load_model()

print(f"Model loaded: {model.is_loaded()}")
print(f"Class labels: {model.class_labels}")

In [ ]:
# Initialize the predictor
predictor = PneumoniaPredictor()

# Check health
health = predictor.health_check()
print(health)

In [ ]:
# Create a sample image for testing
def create_sample_image(size=(500, 720), mode='RGB'):
    """Create a sample image for testing."""
    if mode == 'RGB':
        image_array = np.random.randint(0, 256, (*size[::-1], 3), dtype=np.uint8)
    else:
        image_array = np.random.randint(0, 256, size[::-1], dtype=np.uint8)
    return Image.fromarray(image_array, mode=mode)

# Create and display a sample image
sample_image = create_sample_image()
plt.figure(figsize=(8, 6))
plt.imshow(sample_image)
plt.title("Sample Image for Testing")
plt.axis('off')
plt.show()

print(f"Sample image size: {sample_image.size}")
print(f"Sample image mode: {sample_image.mode}")

In [ ]:
# Test image preprocessing
preprocessed = preprocess_image(sample_image, verbose=True)
print(f"Preprocessed image shape: {preprocessed.shape}")
print(f"Value range: [{preprocessed.min():.3f}, {preprocessed.max():.3f}]")

In [ ]:
# Make a prediction
result = predictor.predict_from_image(sample_image)

print("Prediction Results:")
print(f"Prediction ID: {result['prediction_id']}")
print(f"Predicted Class: {result['predicted_class']}")
print(f"Confidence: {result['confidence']:.3f}")
print(f"All predictions: {[f'{p:.3f}' for p in result['predictions'][0]]}")

In [ ]:
# Visualize prediction probabilities
predictions = result['predictions'][0]
classes = model.class_labels

plt.figure(figsize=(10, 6))
bars = plt.bar(classes, predictions, color=['green', 'orange', 'red'])
plt.title('Pneumonia Prediction Probabilities')
plt.ylabel('Probability')
plt.ylim(0, 1)

# Add value labels on bars
for bar, prob in zip(bars, predictions):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{prob:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# Test API endpoints (if server is running)
def test_api_endpoints():
    """Test the API endpoints if server is running."""
    base_url = "http://localhost:8000/api/v1"
    
    try:
        # Test health endpoint
        response = requests.get(f"{base_url}/health", timeout=5)
        print(f"Health check status: {response.status_code}")
        print(f"Health response: {response.json()}")
        
        # Test info endpoint
        response = requests.get(f"{base_url}/info", timeout=5)
        print(f"Info status: {response.status_code}")
        print(f"Service info: {response.json()}")
        
        return True
        
    except requests.exceptions.RequestException as e:
        print(f"API not available: {e}")
        print("Start the server with: python cmd/server.py")
        return False

# Test the API
api_available = test_api_endpoints()

In [ ]:
# Test prediction API if available
if api_available:
    # Convert sample image to bytes
    img_bytes = BytesIO()
    sample_image.save(img_bytes, format='JPEG')
    img_bytes.seek(0)
    
    # Make API request
    files = {'file': ('test.jpg', img_bytes.getvalue(), 'image/jpeg')}
    response = requests.post("http://localhost:8000/api/v1/predict", files=files)
    
    if response.status_code == 200:
        api_result = response.json()
        print("API Prediction Results:")
        print(f"Predicted Class: {api_result['predicted_class']}")
        print(f"Confidence: {api_result['confidence']:.3f}")
    else:
        print(f"API request failed: {response.status_code}")
        print(response.text)
else:
    print("API not available for testing")